In [1]:
import pandas as pd

In [2]:
def calculate_accuracy(data):
    if 'parsed_answer' not in data.columns or 'correct_answer' not in data.columns:
        raise ValueError("DataFrame must contain 'parsed_answer' and 'correct_answer' columns")
    
    correct_predictions = (data['parsed_answer'] == data['correct_answer']).sum()
    total_predictions = data.shape[0]
    
    accuracy = correct_predictions / total_predictions
    
    return accuracy

# MISTRAL

In [35]:
file_path = '../story_analogies_result.csv'
data = pd.read_csv(file_path, index_col=0)

In [36]:
data

,source_story,correct_analogy,false_analogy,full_prompt,raw_generation,parsed_answer,correct_answer
0,William was a patient in a psychiatric hospita...,Karen always did poorly in high school so she ...,Karen disliked high school so she always had t...,This is a source story:\n\nWilliam was a patie...,"After analyzing the stories, I can conclude th...",A,A
1,Mr. Newton was the manager of a company that m...,Ms. Boyce was in charge of installing heat lam...,Ms. Boyce was in charge of installing sun lamp...,This is a source story:\n\nMr. Newton was the ...,"After analyzing the three stories, I can concl...",A,A
2,Joseph was a millionaire who hired a chauffeur...,Alexander was a man who lived with his wife in...,Many years ago there was man named Alexander w...,This is a source story:\n\nJoseph was a millio...,The stories that are analogous to the source s...,B,A
3,"Karla, an old hawk, lived at the top of a tall...",Once there was a small country called Zerdia t...,Once there was a small country called Zerdia t...,"This is a source story:\n\nKarla, an old hawk,...",The stories that are analogous to the source s...,A,A
4,Peter was a young man from Iceland who moved t...,Quiggly Company was as small company that manu...,Quiggly Company was a small company that manuf...,This is a source story:\n\nPeter was a young m...,The analogous story is:\n\n<ans> A </ans>\n\nI...,A,A
5,Julius was a mule who discovered several pears...,A girl named Cindy found some records she was ...,A girl named Cindy found some records she want...,This is a source story:\n\nJulius was a mule w...,The stories that are analogous to the source s...,A,A
6,Percy the mockingbird spent the whole warm sea...,Sam travelled all over the world buying beauti...,Sam travelled all over the world buying beauti...,This is a source story:\n\nPercy the mockingbi...,The stories A and B are analogous to the sourc...,NaN,A
7,A dog named Leonard was trotting along a dock ...,Charlie was walking through a hotel with an ex...,Charlie was walking through a hotel with an ex...,This is a source story:\n\nA dog named Leonard...,The stories that are analogous to the source s...,A,A
8,Boris and Ivan thought well of one anothers' s...,John and Christine loved each other and decide...,John and Christine loved each other and decide...,This is a source story:\n\nBoris and Ivan thou...,"After analyzing the stories, I found that Stor...",B,A
9,"Two small countries, Bolon and Salam, were adj...","Two sixth-grade boys, Lincoln and Moreland wer...","Two seventh graders, Lincoln and Moreland, wer...",This is a source story:\n\nTwo small countries...,The stories A and B are analogous to the sourc...,NaN,A


In [37]:
print(data.iloc[0]['source_story'])

William was a patient in a psychiatric hospital who was confined indoors almost all the time. He could never pass the monthly room inspections so he hated them. He spent most of his time daydreaming about food.
A few day before the April inspection William's room was still a mess since he had done nothing but daydream. To provide William with an incentive, the nurse promised him some gingerbread from the cookie shop if he scrubbed his room and put it in order once and for all.
William was overjoyed. But there was no longer enough time for him to put it in order. As a result, he did not pass the inspection and did not get any gingerbread.
William sulked all day and slammed his door so hard the plaster cracked, but he still didn't get any gingerbread.


In [38]:
print(data.iloc[0]['correct_analogy'])

Karen always did poorly in high school so she despised it. But she loved vacations. She spent most of her time dreaming about going to Hawaii.
Not long before the end of her fourth year Karen was not doing at all well in her classes because she had spent all her time daydreaming. To motivate her, Karen's father promised her that if she did well enough during the next few weeks to graduate from high school he would pay for her trip to Hawaii.
This made Karen extremely happy. But she was too far behind in her classes. Consequently she failed too many and did not go to Hawaii.


In [39]:
print(data.iloc[0]['false_analogy'])

Karen disliked high school so she always had trouble getting passing grades in her classes. Vacations were her favorite part of the school year.
Not long before the end of her fourth year of school, Karen was not doing well at all because she despised school. To motivate her, Karen's father promised her a trip to Hawaii if she would just pass enough of her classes to graduate.
This made Karen extremely happy. During the last few weeks of school she spent most of her time dreaming about Hawaii and preparing for her trip there. Consequently, she failed to graduate and she did not get to go.


In [40]:
print(data.iloc[0]['full_prompt'])

This is a source story:

William was a patient in a psychiatric hospital who was confined indoors almost all the time. He could never pass the monthly room inspections so he hated them. He spent most of his time daydreaming about food.
A few day before the April inspection William's room was still a mess since he had done nothing but daydream. To provide William with an incentive, the nurse promised him some gingerbread from the cookie shop if he scrubbed his room and put it in order once and for all.
William was overjoyed. But there was no longer enough time for him to put it in order. As a result, he did not pass the inspection and did not get any gingerbread.
William sulked all day and slammed his door so hard the plaster cracked, but he still didn't get any gingerbread.

This is story A:

Karen always did poorly in high school so she despised it. But she loved vacations. She spent most of her time dreaming about going to Hawaii.
Not long before the end of her fourth year Karen was 

In [41]:
print(data.iloc[0]['raw_generation'])

After analyzing the stories, I can conclude that:

<ans> A </ans>

The reason is that both stories A and the source story share the same pattern:

1. The protagonist is not motivated to take action due to a lack of interest in their current situation (high school or hospital life).
2. A promise of a reward (gingerbread or trip to Hawaii) is made to motivate them to take action.
3. Despite the promise, the protagonist fails to take action or achieve their goal due to their lack of motivation or procrastination.

Story B does not fit this pattern, as Karen's father's promise actually motivates her to daydream and prepare for her trip, rather than motivating her to take action in her schoolwork.


In [14]:
data.iloc[0]['parsed_answer']

'A'

In [15]:
data.iloc[0]['correct_answer']

'A'

In [42]:
nan_rows = data[data['parsed_answer'].isna()]
print(nan_rows['full_prompt'])

6     This is a source story:\n\nPercy the mockingbi...
9     This is a source story:\n\nTwo small countries...
12    This is a source story:\n\nKing Otto was a war...
16    This is a source story:\n\nOnce there was a te...
Name: full_prompt, dtype: object


In [44]:
for prompt in nan_rows['raw_generation']:
    print(prompt)
    print("============================================")

The stories A and B are analogous to the source story.

In the source story, Percy the mockingbird visits a squirrel and expects to receive sunflower seeds in return for singing a song, but the squirrel is disappointed and refuses to give him any. Similarly, in story A, Sam visits his mother and expects to receive a loan in return for a gift he bought, but his mother is not pleased and refuses to give him any money.

In story B, Sam visits his mother and is scolded for wasting his time, and then tries to ask for a loan, but his mother still refuses to give him any money. This story also has a similar theme of someone expecting something in return for a gift or service, but being disappointed and rejected.

Therefore, both stories A and B are analogous to the source story.
The stories A and B are analogous to the source story.

In all three stories, a weaker party (Bolon/Salam/Moreland) tries to make the best of a difficult situation by making a deal with a stronger party (Mayonia/Chad)

# LLAMA-3

In [45]:
file_path = '../story_analogies_result_llama3.csv'
data = pd.read_csv(file_path, index_col=0)

In [46]:
print(data.iloc[0]['full_prompt'])

This is a source story:

William was a patient in a psychiatric hospital who was confined indoors almost all the time. He could never pass the monthly room inspections so he hated them. He spent most of his time daydreaming about food.
A few day before the April inspection William's room was still a mess since he had done nothing but daydream. To provide William with an incentive, the nurse promised him some gingerbread from the cookie shop if he scrubbed his room and put it in order once and for all.
William was overjoyed. But there was no longer enough time for him to put it in order. As a result, he did not pass the inspection and did not get any gingerbread.
William sulked all day and slammed his door so hard the plaster cracked, but he still didn't get any gingerbread.

This is story A:

Karen always did poorly in high school so she despised it. But she loved vacations. She spent most of her time dreaming about going to Hawaii.
Not long before the end of her fourth year Karen was 

In [47]:
print(data.iloc[0]['raw_generation'])

The stories that are analogous to the source story are:

<ans> A </ans>

The reason is that in both the source story and story A, the protagonist is motivated by a reward (gingerbread/ trip to Hawaii) but ultimately fails to achieve the goal due to their lack of effort or time. In both cases, the protagonist is distracted by daydreaming and fails to take action to achieve their goal.


In [48]:
nan_rows = data[data['parsed_answer'].isna()]
print(nan_rows['full_prompt'])

7     This is a source story:\n\nA dog named Leonard...
12    This is a source story:\n\nKing Otto was a war...
14    This is a source story:\n\nMark the chicken fa...
Name: full_prompt, dtype: object


In [49]:
for prompt in nan_rows['raw_generation']:
    print(prompt)
    print("============================================")

After analyzing the stories, I can see that both A and B have similar structures and plot elements to the source story.

However, the key similarity is the theme of losing something valuable (a steak or a watch) and then trying to take something else of value, only to end up losing even more.

In both A and B, the protagonist loses something (a steak or a watch) and then tries to take something else, leading to further loss (a tooth or both watches).

So, I would say that both stories A and B are analogous to the source story.

Final answer: <ans> Both A and B </ans>
The stories A and B are analogous to the source story.

Both stories feature a character who is obsessed with a particular activity (astronomy in Cornelius's case, conquest and riches in King Otto's case) and neglects other important aspects of their life (maintaining their home in Cornelius's case, domestic expenditures in King Otto's case). As a result, they suffer an accident that forces them to re-evaluate their priori

# LOGITS

## Mistral

In [15]:
file_path = '../results/story_analogies_far_logits_mistral.csv'
data = pd.read_csv(file_path, index_col=0)

In [16]:
data

,source_story,correct_analogy,false_analogy,full_prompt,raw_generation,parsed_answer,correct_answer,ambiguous
0,William was a patient in a psychiatric hospita...,Karen always did poorly in high school so she ...,Karen disliked high school so she always had t...,This is a source story:\n\nWilliam was a patie...,<ans> A </ans>\n\nIn both the source story and...,A,A,False
1,Mr. Newton was the manager of a company that m...,Ms. Boyce was in charge of installing heat lam...,Ms. Boyce was in charge of installing sun lamp...,This is a source story:\n\nMr. Newton was the ...,<ans> A </ans>\n\nBoth stories describe an inc...,A,A,False
2,Joseph was a millionaire who hired a chauffeur...,Alexander was a man who lived with his wife in...,Many years ago there was man named Alexander w...,This is a source story:\n\nJoseph was a millio...,<ans> A </ans>\n\nIn both the source story and...,A,A,False
3,"Karla, an old hawk, lived at the top of a tall...",Once there was a small country called Zerdia t...,Once there was a small country called Zerdia t...,"This is a source story:\n\nKarla, an old hawk,...",<ans> A </ans>\n\nIn both the source story and...,A,A,False
4,Peter was a young man from Iceland who moved t...,Quiggly Company was as small company that manu...,Quiggly Company was a small company that manuf...,This is a source story:\n\nPeter was a young m...,<ans> A </ans>\n\nExplanation: The source stor...,A,A,False
5,Julius was a mule who discovered several pears...,A girl named Cindy found some records she was ...,A girl named Cindy found some records she want...,This is a source story:\n\nJulius was a mule w...,<ans> A </ans>\n\nBoth stories involve a chara...,A,A,False
6,Percy the mockingbird spent the whole warm sea...,Sam travelled all over the world buying beauti...,Sam travelled all over the world buying beauti...,This is a source story:\n\nPercy the mockingbi...,<ans> B </ans>\n\nExplanation: In both the sou...,B,A,False
7,A dog named Leonard was trotting along a dock ...,Charlie was walking through a hotel with an ex...,Charlie was walking through a hotel with an ex...,This is a source story:\n\nA dog named Leonard...,<ans> A </ans>\n\nThe reason is that in both t...,A,A,False
8,Boris and Ivan thought well of one anothers' s...,John and Christine loved each other and decide...,John and Christine loved each other and decide...,This is a source story:\n\nBoris and Ivan thou...,<ans> A </ans>\n\nIn both the source story and...,A,A,False
9,"Two small countries, Bolon and Salam, were adj...","Two sixth-grade boys, Lincoln and Moreland wer...","Two seventh graders, Lincoln and Moreland, wer...",This is a source story:\n\nTwo small countries...,<ans> A </ans>\n\nThe source story and Story A...,A,A,False


In [19]:
print(data.iloc[6]['raw_generation'])

<ans> B </ans>

Explanation: In both the source story and Story B, the characters visit someone they expect to receive help from, but instead receive criticism or rejection. The source story involves Percy the mockingbird and a squirrel, while Story B involves Sam and his mother. In neither story does the receiving character respond positively to the gift or request for help.</s>


In [18]:
calculate_accuracy(data)

0.9444444444444444

## LLAMA-3

In [9]:
file_path = '../results/story_analogies_far_logits_llama3.csv'
data = pd.read_csv(file_path, index_col=0)

In [10]:
data

,source_story,correct_analogy,false_analogy,full_prompt,raw_generation,parsed_answer,correct_answer,ambiguous
0,William was a patient in a psychiatric hospita...,Karen always did poorly in high school so she ...,Karen disliked high school so she always had t...,This is a source story:\n\nWilliam was a patie...,The analogous story is:\n\n<ans> A </ans>\n\nT...,A,A,False
1,Mr. Newton was the manager of a company that m...,Ms. Boyce was in charge of installing heat lam...,Ms. Boyce was in charge of installing sun lamp...,This is a source story:\n\nMr. Newton was the ...,"After analyzing the three stories, I can concl...",A,A,False
2,Joseph was a millionaire who hired a chauffeur...,Alexander was a man who lived with his wife in...,Many years ago there was man named Alexander w...,This is a source story:\n\nJoseph was a millio...,"After analyzing the three stories, I can concl...",A,A,False
3,"Karla, an old hawk, lived at the top of a tall...",Once there was a small country called Zerdia t...,Once there was a small country called Zerdia t...,"This is a source story:\n\nKarla, an old hawk,...",The stories that are analogous to the source s...,A,A,False
4,Peter was a young man from Iceland who moved t...,Quiggly Company was as small company that manu...,Quiggly Company was a small company that manuf...,This is a source story:\n\nPeter was a young m...,<ans> A </ans>\n\nBoth stories A and B describ...,A,A,False
5,Julius was a mule who discovered several pears...,A girl named Cindy found some records she was ...,A girl named Cindy found some records she want...,This is a source story:\n\nJulius was a mule w...,The stories that are analogous to the source s...,A,A,False
6,Percy the mockingbird spent the whole warm sea...,Sam travelled all over the world buying beauti...,Sam travelled all over the world buying beauti...,This is a source story:\n\nPercy the mockingbi...,The analogous story is <ans> A </ans>. Both st...,A,A,False
7,A dog named Leonard was trotting along a dock ...,Charlie was walking through a hotel with an ex...,Charlie was walking through a hotel with an ex...,This is a source story:\n\nA dog named Leonard...,"After analyzing the three stories, I can see t...",A,A,True
8,Boris and Ivan thought well of one anothers' s...,John and Christine loved each other and decide...,John and Christine loved each other and decide...,This is a source story:\n\nBoris and Ivan thou...,"After analyzing the stories, I can see that bo...",A,A,False
9,"Two small countries, Bolon and Salam, were adj...","Two sixth-grade boys, Lincoln and Moreland wer...","Two seventh graders, Lincoln and Moreland, wer...",This is a source story:\n\nTwo small countries...,"After analyzing the stories, I can see that St...",A,A,False


In [12]:
print(data.iloc[7]['raw_generation'])

After analyzing the three stories, I can see that both stories A and B share similar plot elements with the source story.

* In all three stories, the main character loses something valuable (steak, watch).
* The main character tries to take something similar from someone else (another dog, another man).
* Unfortunately, the attempt ends in failure, with the main character losing something else (a tooth, both watches).

Therefore, I conclude that both stories A and B are analogous to the source story.

Final answer: <ans> A </ans> and <ans> B </ans>


In [14]:
calculate_accuracy(data)

0.9444444444444444

## STARLING

In [9]:
file_path = '../results/story_analogies_far_logits_starling7b-beta.csv'
data = pd.read_csv(file_path, index_col=0)

In [11]:
data

,source_story,correct_analogy,false_analogy,full_prompt,raw_generation,parsed_answer,correct_answer,ambiguous
0,William was a patient in a psychiatric hospita...,Karen always did poorly in high school so she ...,Karen disliked high school so she always had t...,This is a source story:\n\nWilliam was a patie...,\n\n<ans> A </ans>\n\nThe source story and sto...,A,A,False
1,Mr. Newton was the manager of a company that m...,Ms. Boyce was in charge of installing heat lam...,Ms. Boyce was in charge of installing sun lamp...,This is a source story:\n\nMr. Newton was the ...,\n\n<ans> A </ans>\n\nThe source story and sto...,A,A,False
2,Joseph was a millionaire who hired a chauffeur...,Alexander was a man who lived with his wife in...,Many years ago there was man named Alexander w...,This is a source story:\n\nJoseph was a millio...,\n\n<ans> B </ans>\n\nThe source story and sto...,B,A,True
3,"Karla, an old hawk, lived at the top of a tall...",Once there was a small country called Zerdia t...,Once there was a small country called Zerdia t...,"This is a source story:\n\nKarla, an old hawk,...",\n\n<ans> A </ans>\n\nThe source story and sto...,A,A,False
4,Peter was a young man from Iceland who moved t...,Quiggly Company was as small company that manu...,Quiggly Company was a small company that manuf...,This is a source story:\n\nPeter was a young m...,\n\n<ans> B </ans>\n\nThe source story is abou...,B,A,True
5,Julius was a mule who discovered several pears...,A girl named Cindy found some records she was ...,A girl named Cindy found some records she want...,This is a source story:\n\nJulius was a mule w...,\n\n<ans> A </ans>\n\nThe source story and sto...,B,A,True
6,Percy the mockingbird spent the whole warm sea...,Sam travelled all over the world buying beauti...,Sam travelled all over the world buying beauti...,This is a source story:\n\nPercy the mockingbi...,\n\n<ans> B </ans>\n\nThe source story involve...,B,A,False
7,A dog named Leonard was trotting along a dock ...,Charlie was walking through a hotel with an ex...,Charlie was walking through a hotel with an ex...,This is a source story:\n\nA dog named Leonard...,\n\n<ans> B </ans>\n\nThe story B is analogous...,B,A,False
8,Boris and Ivan thought well of one anothers' s...,John and Christine loved each other and decide...,John and Christine loved each other and decide...,This is a source story:\n\nBoris and Ivan thou...,\n\n<ans> A </ans>\n\nThe source story and sto...,B,A,True
9,"Two small countries, Bolon and Salam, were adj...","Two sixth-grade boys, Lincoln and Moreland wer...","Two seventh graders, Lincoln and Moreland, wer...",This is a source story:\n\nTwo small countries...,\n\n<ans> B </ans>\n\nThe story B is analogous...,B,A,False


In [13]:
print(data.iloc[1]['raw_generation'])



<ans> A </ans>

The source story and story A are analogous because in both stories, the manager/inventor and Ms. Boyce are alarmed by an accident and decide to implement a safety feature (safety razor and warning label) immediately after the incident. In story B, the manager/inventor does not consider the warning label necessary until after she is burned, which is not as immediate a response as in story A.

The source story and story A are analogous because in both stories, the manager/inventor and Ms. Boyce are alarmed by an accident and decide to implement a safety feature (safety razor and warning label) immediately after the incident. In story B, the manager/inventor does not consider the warning label necessary until after she is burned, which is not as immediate a response as in story A.

The source story and story A are analogous because in both stories, the manager/inventor and Ms. Boyce are alarmed by an accident and decide to implement a safety feature (safety razor and war